In [73]:
import pandas as pd
import numpy as np

T = 600
P = 0.05
#需要优化的量

#关系矩阵
#dts
#M1=np.zeros((19,106),dtype=int) #进口：对应1
#M3=np.zeros((19,106),dtype=int) #国产：对应2
#ctd
#M2=np.zeros((1,18),dtype=int) #进口
#M4=np.zeros((1,18),dtype=int) #国产

#仓库状态矩阵
#O=np.zeros((20,1),dtype=int)  #是否开仓
#Z=np.zeros((20,1),dtype=int)  #是否智能


In [74]:
#参数导入
#国产，对应2
Q_dm=pd.read_csv('data_p/Q_dm.csv')
Q_dm=Q_dm.to_numpy()
Q_dm=Q_dm[:,1]
Q_dm=Q_dm.reshape((1,106))
Q_dm.shape

(1, 106)

In [75]:
#港口，对应1
Q_im=pd.read_csv('data_p/Q_im.csv')
Q_im=Q_im.to_numpy()
Q_im=Q_im[:,1]
Q_im=Q_im.reshape((1,106))
Q_dm.shape

(1, 106)

In [76]:
L_dts=pd.read_csv('data_p/d_matrix_dts.csv')
L_dts=L_dts.to_numpy()
L_dts=L_dts[1:]
L_dts=L_dts[:,2:]
L_dts_1=np.delete(L_dts,11,axis=0)  #im(delete neilu)
#k = 10 
#row_to_move = L_dts_1[k]
#L_dts_1 = np.delete(L_dts_1, k, axis=0)
#L_dts_1 = np.vstack([row_to_move, L_dts_1])

L_dts_2=np.delete(L_dts,6,axis=0)  #dm(delete gangkou)
#k = 6 
#row_to_move = L_dts_2[k]
#L_dts_2 = np.delete(L_dts_2, k, axis=0)
#L_dts_2 = np.vstack([row_to_move, L_dts_2])
L_dts_2.shape

(19, 106)

In [77]:
T_dts=pd.read_csv('data_p/t_matrix_dts.csv')
T_dts=T_dts.to_numpy()
T_dts=T_dts[1:]
T_dts=T_dts[:,2:]
T_dts_1=np.delete(T_dts,11,axis=0)#im(delete neilu)
T_dts_2=np.delete(T_dts,6,axis=0)#dm(delete gangkou)
T_dts_1

array([[532.0, 167.0, 491.0, ..., 578.0, 1478.0, 364.0],
       [1376.0, 959.0, 1295.0, ..., 556.0, 644.0, 935.0],
       [1639.0, 1209.0, 1163.0, ..., 962.0, 1130.0, 1253.0],
       ...,
       [1169.0, 758.0, 1140.0, ..., 310.0, 877.0, 709.0],
       [1189.0, 753.0, 651.0, ..., 501.0, 1249.0, 816.0],
       [948.0, 406.0, 859.0, ..., 175.0, 1137.0, 440.0]], dtype=object)

In [78]:
L_ctd=pd.read_csv('data_p/d_matrix_ctd.csv')
L_ctd=L_ctd.to_numpy()
L_ctd=L_ctd[:,2:]
L_ctd[0].shape

(18,)

In [79]:
T_ctd=pd.read_csv('data_p/t_matrix_ctd.csv')
T_ctd=T_ctd.to_numpy()
T_ctd=T_ctd[:,2:]

In [80]:
Pf=pd.read_csv('data_p/Pf.csv')
Pf=Pf.to_numpy()
Pf=Pf[:,1]
Pf.shape

(20,)

In [81]:
Of=pd.read_csv('data_p/Of.csv')
Of=Of.to_numpy()
Of=Of[:,1]
#Of.reshape((1,20))
Of.shape

(20,)

In [82]:
Cap=pd.read_csv('data_p/Cap.csv')
Cap=Cap.to_numpy()
Cap=Cap[:,1]
#Cap.reshape((1,20))
Cap.shape

(20,)

In [83]:
#目标优化式
##处置成本
def ccl_cp(Z,Q,M_s,M_d,k,Of,Pf):
    cp = 0
    Q_dc = M_s * Q #19*106 1*106
    Q_dc = Q_dc.sum(axis=1) # dc(rdc+一个cdc)的订单量 19*1
    #print(Q_c) #19,一维数组
    #tmp1=tmp1.reshape((1,19))
    #Q_rdc = np.delete(Q_dc,k) # rdc的订单量 18*1
    #tmp = M_d * Q_rdc # 18*1 18*1
    #ccp = tmp.sum()
    ccp = Q.sum()
    kx = 11 if k == 6 else 6
    Pf = np.delete(Pf,kx) 
    Of = np.delete(Of,kx)
    Z = np.delete(Z,kx)
    m = -0.5*Z+1
    for i in range(0,19):
        cp += Of[i] + Q_dc[i] * Pf[i] * m[i] + Z[i]
    return cp+ccp

#运输成本
def ccl_ct(k,Q,M_s,M_d,L_s,L_d):
    total = 0
    tmp1 = M_s * Q * L_s * 1.25 # 19*106 1*106 19*106
    print("tmp1.shape",tmp1.shape)
    Q_c = Q @ M_s.T # 1*106 106*19 
    print("Q_c.shape",Q_c.shape) # 1*19
    Q_c = np.delete(Q_c,k,axis=1) # 1*18
    tmp2 = M_d * Q_c * L_d.T * 0.6 # 1*18 1*18 1*18
    print("tmp2.shape",tmp2.shape) 
    total += tmp1.sum() + tmp2.sum()
    return total

#总成本
def cal_all(M1,M2,M3,M4,Z,Of,Pf,L_ctd,L_dts_1,L_dts_2):
    M1=M1.tolist()
    M1=np.array(M1)
    M2=M2.tolist()
    M2=np.array(M2)
    M3=M3.tolist()
    M3=np.array(M3)
    M4=M4.tolist()
    M4=np.array(M4)
    #O=O.tolist()
    #O=np.array(O)
    Z=Z.tolist()
    #print(Z[0])
    Z=np.array(Z)
    c=0
    #进口
    c+=ccl_cp(Z,Q_im,M1,M2,6,Of,Pf)
    #print(c)
    c+=ccl_ct(10,Q_im,M1,M2,L_dts_1,L_ctd[0])
    #print(c)
    #国产
    c+=ccl_cp(Z,Q_dm,M3,M4,10,Of,Pf)
    #print(c)
    c+=ccl_ct(6,Q_dm,M3,M4,L_dts_2,L_ctd[1])
    #print(c)
    return c

In [84]:
from gurobipy import Model, GRB, quicksum

# 初始化模型
model = Model('Optimization')

# 定义决策变量
M1 = model.addMVar(shape=(19, 106), vtype=GRB.BINARY, name="M1")
M2 = model.addMVar(shape=(1, 18), vtype=GRB.BINARY, name="M2")
M3 = model.addMVar(shape=(19, 106), vtype=GRB.BINARY, name="M3")
M4 = model.addMVar(shape=(1, 18), vtype=GRB.BINARY, name="M4")
#O = model.addMVar(shape=(20, 1), vtype=GRB.BINARY, name="O")
Z = model.addMVar(shape=(20, 1), vtype=GRB.BINARY, name="Z")

# 定义辅助变量 y_im 和 y_dm
BigM = 100000000
#Q_all=Q_im.sum()+Q_dm.sum()
y_im = model.addMVar(19, vtype=GRB.BINARY, name="y_im")
y_dm = model.addMVar(19, vtype=GRB.BINARY, name="y_dm")
# 定义大M辅助变量 y_bigM
y_bigM = model.addMVar(19, vtype=GRB.BINARY, name="y_bigM")

# 定义约束条件
# 时间约束
#时间约束

tmp1=[j for j in range(19)]
tmp2=[j for j in range(19)]
# 定义辅助变量 tmp1 和 tmp2
for i in range(19):
    tmp1[i] = quicksum(M1[i,j] * T_dts_1[i,j] for j in range(106))
for i in range(19):
    tmp2[i] = quicksum(M3[i,j] * T_dts_2[i,j] for j in range(106))
for i in range(19):
    # tmp < T 时，y = 0
    model.addConstr(y_im[i] <= tmp1[i] / T)
    model.addConstr(y_dm[i] <= tmp2[i] / T)
    # tmp > T 时，y = 1
    model.addConstr(y_im[i] * BigM >= tmp1[i] / T - 1)
    model.addConstr(y_dm[i] * BigM >= tmp2[i] / T - 1)
    #model.addConstr(y_im[i] >= (tmp1[i] / T) - BigM * (1 - y_bigM[i]))
    #model.addConstr(y_dm[i] >= (tmp2[i] / T) - BigM * (1 - y_bigM[i]))

# 定义 Q_out
Q_out = quicksum(y_im[i] * Q_im[0,i] + y_dm[i] * Q_dm[0,i] for i in range(19))
# 添加时效满足率条件
Q_all=Q_im.sum()+Q_dm.sum()
model.addConstr(Q_out <= P * Q_all)

'''
Q_all=Q_im.sum()+Q_dm.sum()
print('Q_all=',Q_all)
def time_point_constraint(model, M_im, M_dm, T_dts_1, T_dts_2, Q_im, Q_dm, Q_all, M):
    tmp1 = M_im * T_dts_1 #19*106 19*106
    tmp2 = M_dm * T_dts_2
    #print(tmp1.shape)
    tmp1_sum = tmp1.sum(axis=0) # 106*1 每个门店国产水果的运输时间
    print(tmp1_sum.shape)
    tmp2_sum = tmp2.sum(axis=0)
    
    # 定义 Q_out 为连续变量，表示满足条件的货物总量
    Q_out = model.addVar(vtype=GRB.CONTINUOUS, name="Q_out")
    # 初始化 Q_out 为 0
    model.addConstr(Q_out == 0)
    
    # 遍历所有门店，添加相应的条件
    for i in range(106):
        # 定义二进制变量 b_i
        b_i = model.addVar(vtype=GRB.BINARY, name=f"b_{i}")
        c_i = model.addVar(vtype=GRB.BINARY, name=f"c_{i}")
        
        # 约束：如果 tmp1_sum[i] >= T，则 b_i = 1
        model.addConstr(tmp1_sum[i] - T >= -(1 - b_i) * M, f"b_cond_1_{i}")
        model.addConstr(tmp1_sum[i] - T >= b_i * M, f"b_cond_2_{i}")
        model.addConstr(tmp2_sum[i] - T <= c_i * M, f"c_cond_1_{i}")
        model.addConstr(tmp2_sum[i] - T >= -(1 - c_i) * M, f"c_cond_2_{i}")
        
        # 约束：如果 tmp1_sum[i] <= T，则 b_i = 1
        model.addConstr(tmp1_sum[i] <= T + (1 - b_i) * M, f"b_cond_1_{i}")
        model.addConstr(tmp1_sum[i] >= T - b_i * M, f"b_cond_2_{i}")
        model.addConstr(tmp2_sum[i] <= T + (1 - c_i) * M, f"c_cond_1_{i}")
        model.addConstr(tmp2_sum[i] >= T - c_i * M, f"c_cond_2_{i}")

        # 通过 b_i 控制 Q_out 的变化
        Q_out += b_i * Q_im[0, i] + c_i * Q_dm[0, i]
        
    out_point = Q_out / Q_all
    return out_point

#out_point = time_point(M1,M3)
out_point = time_point_constraint(model, M1, M3, T_dts_1, T_dts_2, Q_im, Q_dm, Q_all, 10000)
model.addConstr(out_point >= P, name="time_lim_constraint")
'''

# 存储容量约束
for j in range(20):
    if j ==6 or j == 10:
        continue
    elif j < 6:
        model.addConstr(quicksum(M1[j, i] * Q_im[0, i] + M3[j, i] * Q_dm[0, i] for i in range(106)) <= Cap[j], "Cap_Limit_{}".format(j))
    elif 6 < j < 10:
        model.addConstr(quicksum(M1[j, i] * Q_im[0, i] + M3[j-1, i] * Q_dm[0, i] for i in range(106)) <= Cap[j], "Cap_Limit_{}".format(j))
    else:
        model.addConstr(quicksum(M1[j-1, i] * Q_im[0, i] + M3[j-1, i] * Q_dm[0, i] for i in range(106)) <= Cap[j], "Cap_Limit_{}".format(j))


# cdc 必须开
#model.addConstr(O[6] == 1, "CDC_Open_6")
#model.addConstr(O[11] == 1, "CDC_Open_11")

# 每个门店对应一个仓库
model.addConstrs((quicksum(M1[:, i]) == 1 for i in range(106)), "Store_1")
model.addConstrs((quicksum(M3[:, i]) == 1 for i in range(106)), "Store_3")

# 每个rdc对应一个cdc

def rdc_cdc_constraint(model, k, M_1, M_2, M):
    M_1=M_1.tolist()
    M_1=np.array(M_1)
    M_1_dc = np.delete(M_1,k,axis=0)
    for i in range(18):
        # 去除第k行的约束表达式
        # M_1_rdc[i] = quicksum((M_1[j, n] for n in range(106)) for j in range(19) if j != k
        M_1_rdc = quicksum(M_1_dc[i, j] for j in range(106))

        # 约束1: 当 M_1_rdc > 0 时，M_2[0, i] = 1
        model.addConstr(M_2[0, i] * M >= M_1_rdc, f"rdc_cdc_cond1_{i}")
        
        # 约束2: 当 M_1_rdc == 0 时，M_2[0, i] = 0
        model.addConstr(M_2[0, i] <= M_1_rdc, f"rdc_cdc_cond2_{i}")

rdc_cdc_constraint(model, k=6, M_1=M1, M_2=M2, M=1000)
rdc_cdc_constraint(model, k=10, M_1=M3, M_2=M4, M=1000)

#rdc_cdc(6,M1,M2)
#rdc_cdc(10,M3,M4)
'''
##M1 & M2约束
def rdc_cdc_constraint(model, k, M_1, M_2, M):
    for i in range(18):
        # 去除第k行的约束表达式
        M_1_rdc = quicksum(M_1[j, i] for j in range(19) if j != k)
        
        # 定义二进制变量，用于条件判断
        binary_var = model.addVar(vtype=GRB.BINARY, name=f"binary_var_{i}")

        # 约束1: M_1_rdc > 0 implies M_2[0, i] = 1 (实现 M_2[0, i] >= binary_var)
        model.addConstr(M_2[0, i] >= binary_var, f"rdc_cdc_cond1_{i}")

        # 约束2: M_1_rdc <= 0 implies M_2[0, i] = 0 (实现 binary_var <= M_1_rdc / M)
        model.addConstr(binary_var * M >= M_1_rdc, f"rdc_cdc_cond2_{i}")


rdc_cdc_constraint(model, k=5, M_1=M1, M_2=M2, M=1000)
rdc_cdc_constraint(model, k=5, M_1=M3, M_2=M4, M=1000)
'''
# 设置目标函数
model.setObjective(cal_all(M1,M2,M3,M4,Z,Of,Pf,L_ctd,L_dts_1,L_dts_2), GRB.MINIMIZE)

# 求解模型
model.optimize()

# 输出结果
if model.status == GRB.OPTIMAL:
    print('Optimal Objective Value:', model.ObjVal)
    
    # 将结果转为pandas DataFrame
    M1_df = pd.DataFrame(M1.X)
    M2_df = pd.DataFrame(M2.X)
    M3_df = pd.DataFrame(M3.X)
    M4_df = pd.DataFrame(M4.X)
    #O_df = pd.DataFrame(O.X)
    Z_df = pd.DataFrame(Z.X)
    
    # 创建一个Excel writer对象
    with pd.ExcelWriter('optimization_results.xlsx') as writer:
        M1_df.to_excel(writer, sheet_name='M1', index=False)
        M2_df.to_excel(writer, sheet_name='M2', index=False)
        M3_df.to_excel(writer, sheet_name='M3', index=False)
        M4_df.to_excel(writer, sheet_name='M4', index=False)
        #O_df.to_excel(writer, sheet_name='O', index=False)
        Z_df.to_excel(writer, sheet_name='Z', index=False)
        
    print("Results saved to 'optimization_results.xlsx'")
else:
    print('No optimal solution found.')

tmp1.shape (19, 106)
Q_c.shape (1, 19)
tmp2.shape (1, 18)
tmp1.shape (19, 106)
Q_c.shape (1, 19)
tmp2.shape (1, 18)
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 379 rows, 4141 columns and 23642 nonzeros
Model fingerprint: 0x8108bda5
Model has 7844 quadratic objective terms
Variable types: 0 continuous, 4141 integer (4141 binary)
Coefficient statistics:
  Matrix range     [9e-02, 1e+08]
  Objective range  [2e-01, 2e+06]
  QObjective range [5e-02, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 1.590482e+07
Presolve removed 0 rows and 19 columns
Presolve time: 0.07s
Presolved: 8223 rows, 11966 columns, 47172 nonzeros
Variable types: 0 continuous, 11966 integer (11966 binary)

Root relaxation: ob